In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import auc

%matplotlib inline


In [ ]:
pr_df = pd.DataFrame()
for pr in snakemake.input:
    tmp_df = pd.read_csv(pr, sep='\t')
    tmp_df['filename'] = pr
    pr_df = pr_df.append(tmp_df)
pr_df['Sample_Name'] = pr_df['filename'].apply(lambda x: os.path.basename(x).split('.')[0])
pr_df.head()

In [ ]:
samples = pr_df.Sample_Name.unique()
fig, axes = plt.subplots(figsize=(6, 12),
                         nrows=len(samples))
for smp, ax in zip(samples, axes):
    tmp_df = pr_df[pr_df.Sample_Name == smp]
    sns.lineplot(ax=ax,
                 data=tmp_df,
                 x='Recall',
                 y='Precision')
    plt.xlabel('Recall')
    plt.ylabel('Precision (1-FDR)')
    plt.title('Precision-Recall Plot: {}'.format(smp))
    lbl = auc(tmp_df.Recall, tmp_df.Precision)
    plt.text(0.0, 0.9, "AUC = {:.3f}".format(lbl))
plt.savefig("results/plots/bagel_pr_curve.pdf")

In [ ]:
ceg_genes = set()
with open(snakemake.params['ess'], 'rt') as fh:
    for line in fh:
        ceg_genes.add(line.strip())
neg_genes = set()
with open(snakemake.params['neg'], 'rt') as fh:
    for line in fh:
        neg_genes.add(line.strip())
pr_df['Essential'] = pr_df.Gene.apply(
    lambda x: 'Essential' if x in ceg_genes 
    else 'Negative' if x in neg_genes else 'Other')

In [ ]:
fig, axes = plt.subplots(figsize=(6, 12),
                         nrows=len(samples))
for smp, ax in zip(samples, axes):
    tmp_df = pr_df[pr_df.Sample_Name == smp]
    sns.displot(data=tmp_df,
                x='BF',
                hue='Essential',
                kde=True,
                hue_order=['Essential', 'Negative'])
    plt.title(smp)
plt.savefig("results/plots/bagel_ess_neg_hist.pdf")

In [ ]:
fig, axes = plt.subplots(figsize=(6, 12),
                         nrows=len(samples))
for smp, ax in zip(samples, axes):
    tmp_df = pr_df[pr_df.Sample_Name == smp]
    sns.kdeplot(ax=ax,
                data=tmp_df,
                x='BF',
                hue='Essential',
                fill=True,
                hue_order=['Essential', 'Negative'])
    ax.set_title(smp)
plt.savefig("results/plots/bagel_ess_neg_dist.pdf")

In [ ]:
fig, axes = plt.subplots(figsize=(6, 12),
                         nrows=len(samples))
for smp, ax in zip(samples, axes):
    tmp_df = pr_df[pr_df.Sample_Name == smp]
    sns.kdeplot(ax=ax,
                data=tmp_df,
                x='BF',
                hue='Essential',
                fill=True,
                hue_order=['Essential', 'Negative'])
    ax.set_title(smp)